#  Load Hitters

In [1]:
import pandas as pd
from sqlalchemy import create_engine

import os
from dotenv import load_dotenv
import psycopg2

from sim import Player

load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASS"),
    port=os.getenv("DB_PORT"),
    sslmode=os.getenv("DB_SSLMODE", "require"),
    sslrootcert=os.getenv("DB_SSLROOTCERT")
)

# Enable SSL but skip server certificate verification (rejectUnauthorized = 0 equivalent)
# conn_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?sslmode=require"
# engine = create_engine(conn_str)

sql1 = """
SELECT *
FROM dev_bwoodrum.ws_2025_hit
"""

# with engine.connect() as conn:
#     hit_df = pd.read_sql_query(sql1, conn)

hit_df = pd.read_sql_query(sql1, conn)

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
output_dir = os.path.join(project_root, "ws_2025_monte_carlo/data")
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "hitter_probs.csv")
# hit_df.to_csv(output_path, index=False)
# print(f"✅ Saved hitter_df to {output_path}")

# --- Convert hitter DataFrame to Player objects ---
hitters = []
for _, row in hit_df.iterrows():
    hitters.append(
        Player(
            name=row["full_name"],         # full name
            hand=row.get("bats", "R"),     # handedness
            probs=row.to_dict(),           # use full row as probability lookup dict
            contact=row.get("contact", 0.0),
            power=row.get("power", 0.0),
            speed=row.get("speed", 0.0),
        )
    )

print(f"✅ Loaded {len(hitters)} hitters into Player objects.")
print(hitters[:5])


✅ Loaded 55 hitters into Player objects.
[Player(Shohei Ohtani, L, contact=0.0, power=0.0, speed=0.0), Player(George Springer, R, contact=0.0, power=0.0, speed=0.0), Player(George Springer, R, contact=0.0, power=0.0, speed=0.0), Player(Max Muncy, L, contact=0.0, power=0.0, speed=0.0), Player(Vladimir Guerrero Jr., R, contact=0.0, power=0.0, speed=0.0)]


C:\Users\Bradley\AppData\Local\Temp\ipykernel_18464\1341583991.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hit_df = pd.read_sql_query(sql1, conn)


# QA Hitters

In [2]:
# Inspect the result
hit_df.head()


,season,bpid,mlbam_id,full_name,pit_hand,bats,pa,out_rate_pred,so_rate_pred,bb_rate_pred,hbp_rate_pred,roe_rate_pred,single_rate_pred,double_rate_pred,triple_rate_pred,hr_rate_pred,drc_plus
0,2025,111306,660271,Shohei Ohtani,R,L,481,0.3397,0.2436,0.1569,0.0043,0.0030,0.1045,0.0328,0.0082,0.1070,211.6265
1,2025,65992,543807,George Springer,L,R,147,0.4124,0.1997,0.1304,0.0092,0.0052,0.1446,0.0422,0.0018,0.0545,154.7330
2,2025,65992,543807,George Springer,R,R,432,0.4244,0.1988,0.1100,0.0145,0.0051,0.1488,0.0405,0.0019,0.0559,150.6533
3,2025,100007,571970,Max Muncy,R,L,308,0.4033,0.2063,0.1637,0.0105,0.0032,0.1196,0.0359,0.0038,0.0538,150.6381
4,2025,107184,665489,Vladimir Guerrero Jr.,L,R,159,0.4689,0.1437,0.1297,0.0071,0.0057,0.1562,0.0478,0.0015,0.0392,143.9738


In [3]:
hit_df.columns

Index(['season', 'bpid', 'mlbam_id', 'full_name', 'pit_hand', 'bats', 'pa',
       'out_rate_pred', 'so_rate_pred', 'bb_rate_pred', 'hbp_rate_pred',
       'roe_rate_pred', 'single_rate_pred', 'double_rate_pred',
       'triple_rate_pred', 'hr_rate_pred', 'drc_plus'],
      dtype='object')

# Load Pitchers

In [3]:
import os
import pandas as pd
from sim import Pitcher 

# --- Load from database ---
sql2 = """
SELECT *
FROM dev_bwoodrum.ws_2025_pit
"""
pit_df = pd.read_sql_query(sql2, conn)

# --- Save to CSV (optional) ---
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
output_dir = os.path.join(project_root, "ws_2025_monte_carlo/data")
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "pitcher_probs.csv")
pit_df.to_csv(output_path, index=False)
print(f"✅ Saved pitcher_df to {output_path}")

# --- Build Pitcher objects ---
pitchers = []
for _, row in pit_df.iterrows():
    pitchers.append(
        Pitcher(
            name=row["full_name"],
            hand=row["pit_hand"],
            dra_minus=row.get("dra_minus", 100),
            dra_minus_L=row.get("dra_minus_L", row.get("dra_minus", 100)),
            dra_minus_R=row.get("dra_minus_R", row.get("dra_minus", 100)),
        )
    )

print(f"✅ Loaded {len(pitchers)} pitchers")
print(pitchers[:5])  # preview first few objects


✅ Saved pitcher_df to c:\Users\Bradley\Documents\GitHub\ws_2025_monte_carlo/data\pitcher_probs.csv
✅ Loaded 60 pitchers
[Pitcher(Justin Wrobleski, L, DRA-=57.2485, L=57.2485, R=57.2485), Pitcher(Blake Snell, L, DRA-=59.9064, L=59.9064, R=59.9064), Pitcher(Alex Vesia, L, DRA-=62.2288, L=62.2288, R=62.2288), Pitcher(Yoshinobu Yamamoto, R, DRA-=62.3736, L=62.3736, R=62.3736), Pitcher(Louis Varland, R, DRA-=65.6770512195122, L=65.6770512195122, R=65.6770512195122)]


C:\Users\Bradley\AppData\Local\Temp\ipykernel_18464\3812836045.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pit_df = pd.read_sql_query(sql2, conn)


# QA Pitchers

In [ ]:
pit_df.columns

Index(['season', 'bpid', 'mlbam_id', 'full_name', 'pit_hand', 'bats', 'bf',
       'out_rate_pred', 'so_rate_pred', 'bb_rate_pred', 'hbp_rate_pred',
       'roe_rate_pred', 'single_rate_pred', 'double_rate_pred',
       'triple_rate_pred', 'hr_rate_pred', 'dra_minus'],
      dtype='object')

In [ ]:
import os
print(os.getcwd())


c:\Users\Bradley\Documents\GitHub\ws_2025_monte_carlo


In [ ]:
from sim.load_data import load_hitters, load_pitchers

hitters = load_hitters("data/hitter_probs.csv")
pitchers = load_pitchers("data/pitcher_probs.csv")

print(hitters[:5])
print(pitchers[:5])

[Shohei Ohtani (L), George Springer (R), George Springer (R), Max Muncy (L), Vladimir Guerrero Jr. (R)]
[Justin Wrobleski (L), Blake Snell (L), Alex Vesia (L), Yoshinobu Yamamoto (R), Louis Varland (R)]
